<a href="https://colab.research.google.com/github/bascoul/TP_Deep_Learning/blob/master/RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="center">
  <td align="center"><a target="_blank" href="http://introtodeeplearning.com">
        <img src="http://introtodeeplearning.com/images/colab/mit.png" style="padding-bottom:5px;" />
      Visit MIT Deep Learning</a></td>
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/aamini/introtodeeplearning/blob/master/lab3/RL.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/aamini/introtodeeplearning/blob/master/lab3/RL.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

# Copyright Information

In [0]:
# Copyright 2020 MIT 6.S191 Introduction to Deep Learning. All Rights Reserved.
# 
# Licensed under the MIT License. You may not use this file except in compliance
# with the License. Use and/or modification of this code outside of 6.S191 must
# reference:
#
# © MIT 6.S191: Introduction to Deep Learning
# http://introtodeeplearning.com
#

# Laboratoire 3 : Apprentissage par renforcement

L'apprentissage par renforcement (RL) est un sous-ensemble de l'apprentissage machine qui pose des problèmes d'apprentissage sous forme d'interactions entre les agents et les environnements. Il suppose souvent que les agents n'ont aucune connaissance préalable du monde, ils doivent donc apprendre à naviguer dans les environnements en optimisant une fonction de récompense. Dans un environnement, un agent peut prendre certaines actions et recevoir un retour d'information, sous forme de récompenses positives ou négatives, concernant sa décision. En tant que tel, la boucle de rétroaction d'un agent s'apparente quelque peu à la notion d'"essai et erreur", ou à la manière dont un enfant pourrait apprendre à distinguer les "bonnes" et les "mauvaises" actions.

En termes pratiques, notre agent RL interagira avec l'environnement en effectuant une action à chaque étape, en recevant une récompense correspondante et en mettant à jour son état en fonction de ce qu'il a "appris".  

! [texte alternatif](https://www.kdnuggets.com/images/reinforcement-learning-fig1-700.jpg)

Alors que le but ultime de l'apprentissage par renforcement est d'apprendre aux agents à agir dans le monde réel et physique, les jeux constituent un terrain d'essai pratique pour le développement d'algorithmes et d'agents RL. Les jeux ont certaines propriétés qui les rendent particulièrement adaptés à la réalité virtuelle : 

1.   Dans de nombreux cas, les jeux ont des environnements parfaitement descriptibles. Par exemple, toutes les règles des échecs peuvent être formellement écrites et programmées dans un simulateur de jeu d'échecs ;
2.   Les parties sont massivement parallélisables. Comme elles ne nécessitent pas d'être exécutées dans le monde réel, les environnements simultanés peuvent être exécutés sur de grands ensembles de données ; 
3.   Des scénarios plus simples dans les jeux permettent un prototypage rapide. Cela accélère le développement d'algorithmes qui pourraient éventuellement être exécutés dans le monde réel ; et
4. ... Les jeux sont amusants ! 

Dans les laboratoires précédents, nous avons exploré les tâches d'apprentissage supervisées (avec les LSTM, les CNN) et non supervisées / semi-supervisées (avec les VAE). L'apprentissage de renforcement est fondamentalement différent, dans la mesure où nous formons un algorithme d'apprentissage profond pour régir les actions de notre agent RL, c'est-à-dire essayer, dans son environnement, de trouver la manière optimale d'atteindre un objectif. L'objectif de la formation d'un agent RL est de déterminer la meilleure étape à suivre pour obtenir le meilleur rendement final. Dans ce laboratoire, nous nous concentrons sur la construction d'un algorithme d'apprentissage de renforcement pour maîtriser deux environnements différents avec une complexité variable. 

1.   **Cartpole** :   Équilibrer un poteau, qui dépasse d'un chariot, en position verticale en déplaçant seulement la base vers la gauche ou la droite. Environnement avec un espace d'observation de faible dimension.
2.   [**Pong**](https://en.wikipedia.org/wiki/Pong) : Battez vos concurrents (qu'il s'agisse d'autres IA ou d'humains !) au jeu de Pong. Environnement avec un espace d'observation à haute dimension - apprentissage direct à partir de pixels bruts.

Commençons ! Tout d'abord, nous allons importer TensorFlow, le package de cours et quelques dépendances.


In [0]:
!apt-get install -y xvfb python-opengl x11-utils > /dev/null 2>&1
!pip install gym pyvirtualdisplay scikit-video > /dev/null 2>&1

%tensorflow_version 2.x
import tensorflow as tf

import numpy as np
import base64, io, time, gym
import IPython, functools
import matplotlib.pyplot as plt
from tqdm import tqdm

!pip install mitdeeplearning
import mitdeeplearning as mdl

Avant de nous lancer, prenons un peu de recul et exposons notre approche, qui s'applique généralement aux problèmes d'apprentissage par renforcement en général :

1. **Initialiser notre environnement et notre agent** : nous allons ici décrire les différentes observations et actions que l'agent peut faire dans l'environnement.
2. **Définir la mémoire de notre agent** : cela permettra à l'agent de se souvenir de ses actions, observations et récompenses passées.
3. **Définir une fonction de récompense** : décrit la récompense associée à une action ou une séquence d'actions.
4. **Définir l'algorithme d'apprentissage** : il sera utilisé pour renforcer les bons comportements de l'agent et décourager les mauvais comportements.


# Part 1: Cartpole

## 3.1 Définir l'environnement et l'agent Cartpole

### Environnement 

Afin de modéliser l'environnement pour les tâches de Cartpole et de Pong, nous utiliserons une boîte à outils développée par OpenAI appelée [OpenAI Gym] (https://gym.openai.com/). Il fournit plusieurs environnements prédéfinis pour la formation et le test des agents d'apprentissage par renforcement, y compris ceux pour les tâches classiques de contrôle de la physique, les jeux vidéo Atari et les simulations robotiques. Pour accéder à l'environnement Cartpole, nous pouvons utiliser `env = gym.make("CartPole-v0")`, auquel nous avons eu accès lorsque nous avons importé le paquet `gym`. Nous pouvons instancier différents [environnements] (https://gym.openai.com/envs/#classic_control) en passant le nom de l'environnement à la fonction "make".

Un problème que nous pourrions rencontrer lors du développement des algorithmes RL est que de nombreux aspects du processus d'apprentissage sont intrinsèquement aléatoires : l'initialisation des états du jeu, les changements dans l'environnement et les actions de l'agent. En tant que tel, il peut être utile de définir une "semence" initiale pour l'environnement afin d'assurer un certain niveau de reproductibilité. Tout comme vous pourriez utiliser "numpy.random.seed", nous pouvons appeler la fonction comparable dans gym, "seed", avec notre environnement défini pour assurer que les variables aléatoires de l'environnement sont initialisées de la même façon à chaque fois.

In [0]:
### Instantier l'environment Cartpole ###

env = gym.make("CartPole-v0")
env.seed(1)

Dans Cartpole, un poteau est fixé par une articulation non actionnée à un chariot, qui se déplace sur une piste sans frottement. La perche démarre à la verticale, et le but est de l'empêcher de tomber. Le système est contrôlé en appliquant une force de +1 ou -1 sur le chariot. Une récompense de +1 est prévue pour chaque fois que la perche reste en position verticale. L'épisode se termine lorsque la perche se trouve à plus de 15 degrés de la verticale, ou lorsque le chariot se déplace de plus de 2,4 unités par rapport au centre de la piste. Un résumé visuel de l'environnement du chariot est présenté ci-dessous :

<img width="400px" src="https://danielpiedrahita.files.wordpress.com/2017/02/cart-pole.png"></img>

Compte tenu de cette configuration de l'environnement et de l'objectif du jeu, nous pouvons réfléchir : 1) quelles observations permettent de définir l'état de l'environnement ; 2) quelles actions l'agent peut prendre. 

Tout d'abord, considérons l'espace d'observation. Dans cet environnement Cartpole, nos observations sont :

1. Position du chariot
2. Vitesse du chariot
3. Angle du pôle
4. Taux de rotation des pôles

Nous pouvons confirmer la taille de l'espace en interrogeant l'espace d'observation de l'environnement :


In [0]:
n_observations = env.observation_space
print("Environment has observation space =", n_observations)

Deuxièmement, nous considérons l'espace d'action. A chaque pas de temps, l'agent peut se déplacer à droite ou à gauche. Là encore, nous pouvons confirmer la taille de l'espace d'action en interrogeant l'environnement :

In [0]:
n_actions = env.action_space.n
print("Number of possible actions that the agent can choose from =", n_actions)

### Agent du cartpole

Maintenant que nous avons instancié l'environnement et compris la dimensionnalité des espaces d'observation et d'action, nous sommes prêts à définir notre agent. Dans l'apprentissage en renforcement profond, un réseau neural profond définit l'agent. Ce réseau prendra comme entrée une observation de l'environnement et sortira la probabilité de prendre chacune des actions possibles. Puisque Cartpole est défini par un espace d'observation de faible dimension, un simple réseau neuronal de feed-forward devrait bien fonctionner pour notre agent. Nous allons définir cela en utilisant l'API "séquentielle".


In [0]:
### Definir l'agent Cartpole ###

# Définition d'un réseau neuronal feed-forward
def create_cartpole_model():
  model = tf.keras.models.Sequential([
      # Première couche dense
      tf.keras.layers.Dense(units=32, activation='relu'),

      # A FAIRE : Définir la dernière couche Dense, qui fournira la sortie du réseau.
      # Pensez à l'espace dans lequel l'agent doit agir !
      '''A FAIRE : Couche dense pour produire des probabilités d'action'''
  ])
  return model

cartpole_model = create_cartpole_model()

Maintenant que nous avons défini l'architecture du réseau principal, nous allons définir une *fonction d'action* qui exécute un passage en avant à travers le réseau, en fonction d'un ensemble d'observations et d'échantillons de la sortie. Cet échantillonnage des probabilités de sortie sera utilisé pour sélectionner l'action suivante pour l'agent. 

**Nous utiliserons cette fonction pour Cartpole et Pong, et elle est également applicable à d'autres tâches RL !

In [0]:
### Définir la fonction d'action de l'agent ###

# Fonction qui prend les observations en entrée, exécute un modèle de passage en avant 
#   et produit une action échantillonnée.
# Arguments :
#   model: le réseau qui définit notre agent
#   observation: l'observation qui alimente le modèle
# Retours :
#   action: choix de l'action de l'agent
def choose_action(model, observation):
  # ajouter une dimension de lot à l'observation
  observation = np.expand_dims(observation, axis=0)

  '''A FAIRE : alimenter les observations à travers le modèle pour prédire les probabilités logarithmiques de chaque action possible.'''
  logits = model.predict('''A FAIRE''')
  
  # faire passer les probabilités logarithmiques par une softmax pour calculer les probabilités réelles
  prob_weights = tf.nn.softmax(logits).numpy()
  
  '''A FAIRE : prélever au hasard de prob_weights pour choisir une action.
  Conseil : examinez attentivement la dimensionnalité des probabilités d'entrée (vecteur) et l'action de sortie (scalaire)'''
  action = np.random.choice('''A FAIRE''', size=1, p=''''A FAIRE''')['''A FAIRE''']

  return action

## 3.2 Définir la mémoire de l'agent

Maintenant que nous avons instancié l'environnement et défini l'architecture du réseau d'agents et la fonction d'action, nous sommes prêts à passer à l'étape suivante de notre flux de travail RL :
1. **Initialiser notre environnement et notre agent** : nous allons ici décrire les différentes observations et actions que l'agent peut effectuer dans l'environnement.
2. **Définir la mémoire de notre agent** : cela permettra à l'agent de se souvenir de ses actions, observations et récompenses passées.
3. **Définir l'algorithme d'apprentissage** : cela sera utilisé pour renforcer les bons comportements de l'agent et décourager les mauvais comportements.

Dans l'apprentissage par renforcement, l'entraînement se fait parallèlement à l'action de l'agent dans l'environnement ; un *épisode* fait référence à une séquence d'actions qui se termine dans un état terminal, comme la chute d'un poteau ou le crash d'un chariot. L'agent devra se souvenir de toutes ses observations et actions, de sorte qu'une fois l'épisode terminé, il puisse apprendre à "renforcer" les bonnes actions et à punir les actions indésirables par le biais de l'apprentissage. Notre première étape consiste à définir une simple mémoire tampon qui contient les observations et les actions de l'agent et les récompenses reçues pour un épisode donné. 

**Une fois de plus, notez la modularité de ce tampon mémoire - il peut être et sera appliqué à d'autres tâches de RL également !

In [0]:
### Mémoire de l'Agent ###

class Memory:
  def __init__(self): 
      self.clear()

  # Remettre à zéro/Redémarrer le buffer de mémoire
  def clear(self): 
      self.observations = []
      self.actions = []
      self.rewards = []

  # Ajouter des observations, des actions, des récompenses à la mémoire
  def add_to_memory(self, new_observation, new_action, new_reward): 
      self.observations.append(new_observation)
      '''A FAIRE : mettre à jour la liste des actions avec de nouvelles actions'''
      # A FAIRE : your update code here
      '''A FAIRE : mettre à jour la liste des récompenses avec de nouvelles récompenses'''
      # A FAIRE : your update code here
        
memory = Memory()

## 3.3 Fonction de récompense

Nous sommes presque prêts à commencer l'algorithme d'apprentissage pour notre agent ! L'étape suivante consiste à calculer les récompenses de notre agent lorsqu'il agit dans l'environnement. Comme nous (et l'agent) ne savons pas si et quand le jeu ou la tâche se terminera (c'est-à-dire quand le poteau tombera), il est utile de mettre l'accent sur l'obtention de récompenses **maintenant** plutôt que plus tard dans le futur -- c'est l'idée de la remise. Il s'agit d'un concept similaire à celui de la remise d'argent dans le cas d'intérêts. Comme dans la conférence, nous utilisons la remise de récompense pour donner plus de préférence à l'obtention de récompenses maintenant plutôt que plus tard dans l'avenir. L'idée de la remise de récompenses est similaire à celle de la remise d'argent dans le cas d'intérêts.

Pour calculer la récompense cumulative attendue, connue sous le nom de **retour**, à un moment donné d'un épisode d'apprentissage, nous additionnons les récompenses escomptées à ce moment $t$, dans un épisode d'apprentissage, et projetées dans le futur. Nous définissons le retour (récompense cumulative) à un pas de temps $t$, $R_{t}$ comme :

>$R_{t}=\sum_{k=0}^\infty\gamma^kr_{t+k}$

où 0 < \gamma < 1$ est le facteur d'actualisation et $r_{t}$ est la récompense au pas de temps $t$, et l'indice $k$ incrémente la projection dans le futur au cours d'un seul épisode d'apprentissage. Intuitivement, vous pouvez considérer que cette fonction déprécie toute récompense reçue aux étapes ultérieures, ce qui obligera l'agent à donner la priorité à l'obtention de récompenses dès maintenant. Comme nous ne pouvons pas étendre les épisodes à l'infini, en pratique, le calcul sera limité au nombre de pas de temps dans un épisode -- après quoi la récompense est supposée être nulle.

Prenez note de la forme de cette somme -- nous devrons être astucieux sur la façon dont nous mettons en œuvre cette fonction. Plus précisément, nous devrons initialiser un tableau de zéros, avec la longueur du nombre de pas de temps, et le remplir avec les valeurs réelles des récompenses actualisées au fur et à mesure que nous parcourons les récompenses de l'épisode, qui auront été enregistrées dans la mémoire de l'agent. En fin de compte, ce qui nous intéresse, c'est de savoir quelles actions sont meilleures que les autres prises dans cet épisode. Nous normaliserons donc nos récompenses calculées, en utilisant la moyenne et l'écart-type des récompenses de l'épisode d'apprentissage.


In [0]:
### Fonction de récompense ###

# Fonction d'aide qui normalise un np.array x
def normalize(x):
  x -= np.mean(x)
  x /= np.std(x)
  return x.astype(np.float32)

# Calculer les récompenses normalisées, actualisées et cumulatives (c'est-à-dire le retour)
# Arguments :
#   rewards: récompense à chaque pas de temps dans l'épisode
#   gamma: facteur d'actualisation
# Retours :
#   récompense réduite normalisée
def discount_rewards(rewards, gamma=0.95): 
  discounted_rewards = np.zeros_like(rewards)
  R = 0
  for t in reversed(range(0, len(rewards))):
      # mettre à jour le montant total de la récompense réduite
      R = R * gamma + rewards[t]
      discounted_rewards[t] = R
      
  return normalize(discounted_rewards)

## 3.4 Algorithme d'apprentissage

Nous pouvons maintenant commencer à définir l'algorithme d'apprentissage qui sera utilisé pour renforcer les bons comportements de l'agent et décourager les mauvais comportements. Dans ce laboratoire, nous nous concentrerons sur les méthodes de *gradient de politique* qui visent à **maximiser** la probabilité d'actions qui se traduisent par des récompenses importantes. De même, cela signifie que nous voulons **minimiser** la probabilité négative de ces mêmes actions. Nous y parvenons simplement en **mettant à l'échelle** les probabilités en fonction des récompenses qui leur sont associées - en amplifiant effectivement la probabilité d'actions qui donnent lieu à des récompenses importantes.

Comme la fonction logarithmique augmente de façon monotone, cela signifie que minimiser **la probabilité négative** est équivalent à minimiser **la probabilité logarithmique négative**.  Rappelons que nous pouvons facilement calculer la log-vraisemblance négative d'une action discrète en évaluant son [entropie croisée softmax] (https://www.tensorflow.org/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits). Comme dans l'apprentissage supervisé, nous pouvons utiliser des méthodes de descente de gradient stochastique pour atteindre la minimisation souhaitée. 

Commençons par définir la fonction de perte.

In [0]:
### Fonction de perte ###

# Arguments :
#   logits: les prévisions du réseau concernant les mesures à prendre
#   actions: les actions prises par l'agent dans un épisode
#   rewards: les récompenses que l'agent a reçues dans un épisode
# Retours :
#   perte
def compute_loss(logits, actions, rewards): 
  '''A FAIRE : compléter la fonction call pour calculer les probabilités de log négatif'''
  neg_logprob = tf.nn.sparse_softmax_cross_entropy_with_logits(logits='''A FAIRE''', labels='''A FAIRE''')
  
  '''A FAIRE : échelonner la probabilité logarithmique négative en fonction des récompenses'''
  loss = tf.reduce_mean('''A FAIRE''')
  return loss

Utilisons maintenant la fonction de perte pour définir une étape d'apprentissage de notre algorithme d'apprentissage :

In [0]:
### Étape d'apprentissage (forward et backpropagation) ###

def train_step(model, optimizer, observations, actions, discounted_rewards):
  with tf.GradientTape() as tape:
      # Propager en avant à travers le réseau de l'agent
      logits = model(observations)

      '''A FAIRE : appeler la fonction compute_loss pour calculer la perte'''
      loss = compute_loss('''A FAIRE''', '''A FAIRE''', '''A FAIRE''')

  '''A FAIRE : effectuer une backpropagation pour minimiser les pertes en utilisant la méthode tape.gradient.
      Utiliser `model.trainable_variables`'''
  grads = tape.gradient('''A FAIRE''', model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))


## 3.5 Lancer Cartpole !

N'ayant aucune connaissance préalable de l'environnement, l'agent commencera à apprendre comment équilibrer le poteau du chariot en se basant uniquement sur les réactions reçues de l'environnement ! Après avoir défini comment notre agent peut se déplacer, comment il prend en compte les nouvelles observations, et comment il met à jour son état, nous verrons comment il apprend progressivement une politique d'actions pour optimiser l'équilibrage du pôle le plus longtemps possible. Pour ce faire, nous suivrons l'évolution des récompenses en fonction de la formation : comment les récompenses devraient-elles évoluer au fur et à mesure de la formation ?

In [0]:
### Apprentissage du Cartpole ! ###

# Taux d'apprentissage et optimiseur
learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate)

# instantier l'agent cartpole
cartpole_model = create_cartpole_model()

# pour tracker notre progression
smoothed_reward = mdl.util.LossHistory(smoothing_factor=0.9)
plotter = mdl.util.PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Rewards')

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists
for i_episode in range(500):

  plotter.plot(smoothed_reward.get())

  # Relancer l'environment
  observation = env.reset()
  memory.clear()

  while True:
      # à partir de notre observation, choisir une action et l'entreprendre dans l'environnement
      action = choose_action(cartpole_model, observation)
      next_observation, reward, done, info = env.step(action)
      # ajouter à la mémoire
      memory.add_to_memory(observation, action, reward)
      
      # L'épisode est-il terminé ? Vous êtes-vous planté ou avez-vous si bien réussi que vous avez terminé ?
      if done:
          # déterminer la récompense totale et en conserver une trace
          total_reward = sum(memory.rewards)
          smoothed_reward.append(total_reward)
          
          # lancer l'apprentissage - n'oubliez pas que nous ne savons rien sur la façon dont l'agent 
          #   se comporte jusqu'à ce qu'il se crashe !
          train_step(cartpole_model, optimizer, 
                     observations=np.vstack(memory.observations),
                     actions=np.array(memory.actions),
                     discounted_rewards = discount_rewards(memory.rewards))
          
          # nettoyer la mémoire
          memory.clear()
          break
      # mettre à jour nos observatons
      observation = next_observation

Pour avoir une idée de la façon dont notre agent s'est acquitté de sa tâche, nous pouvons enregistrer une vidéo du modèle formé qui travaille à l'équilibrage du poteau. Réalisez qu'il s'agit d'un tout nouvel environnement que l'agent n'a jamais vu auparavant !

Affichez la vidéo sauvegardée pour voir comment notre agent s'en est sorti !


In [0]:
saved_cartpole = mdl.lab3.save_video_of_model(cartpole_model, "CartPole-v0")
mdl.lab3.play_video(saved_cartpole)

Quel est le rôle de l'agent ? Pourriez-vous le former pendant une période plus courte tout en obtenant de bons résultats ? Pensez-vous qu'une formation plus longue serait encore plus utile ? 

#Part 2: Pong

Dans Cartpole, nous avions affaire à un environnement statique - en d'autres termes, il ne changeait pas avec le temps. Que se passe-t-il si notre environnement est dynamique et imprévisible ? C'est exactement le cas dans [Pong] (https://en.wikipedia.org/wiki/Pong), puisqu'une partie de l'environnement est le joueur adverse. Nous ne savons pas comment notre adversaire va agir ou réagir à nos actions, donc la complexité de notre problème augmente. Il devient également beaucoup plus intéressant, puisque nous pouvons nous affronter pour battre notre adversaire. RL fournit un cadre puissant pour l'entraînement des systèmes d'IA avec la capacité de gérer et d'interagir avec des environnements dynamiques et imprévisibles. Dans cette partie du laboratoire, nous utiliserons les outils et le flux de travail que nous avons explorés dans la première partie pour construire un agent RL capable de jouer au Pong.


## 3.6 Définir et inspecter l'environnement de Pong

Comme pour Cartpole, nous allons instancier l'environnement Pong dans OpenAI gym, en utilisant une graine de 1.

In [0]:
env = gym.make("Pong-v0", frameskip=5)
env.seed(1); # pour la reproductibilité

Examinons maintenant l'espace d'observation de l'environnement Pong. Dans le cas de Pong, au lieu de quatre descripteurs physiques pour le réglage de Cartpole, nos observations sont les images vidéo individuelles (c'est-à-dire les images) qui décrivent l'état de la carte. Ainsi, les observations sont des images RGB 210x160 (tableaux de forme (210,160,3)).

Nous pouvons à nouveau confirmer la taille de l'espace d'observation par une interrogation :

In [0]:
print("Environment has observation space =", env.observation_space)

Dans Pong, à chaque pas de temps, l'agent (qui contrôle la raquette) a le choix entre six actions : no-op (pas d'opération), move right, move left, fire, fire right, and fire left. Vérifions la taille de l'espace d'action en interrogeant l'environnement :

In [0]:
n_actions = env.action_space.n
print("Number of possible actions that the agent can choose from =", n_actions)

## 3.7 Définir l'agent de Pong

Comme auparavant, nous utiliserons un réseau de neurones pour définir notre agent. Selon vous, quelle architecture de réseau serait particulièrement adaptée à ce jeu ? Comme nos observations se présentent maintenant sous forme d'images, nous ajouterons des couches convolutionnelles au réseau pour augmenter la capacité d'apprentissage de notre réseau.

In [0]:
### Définir l'agent Pong ###

# Définir les couches de manière fonctionnelle pour plus de commodité
# Toutes les couches convolutionnelles seront dotées de l'activation ReLu
Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu')
Flatten = tf.keras.layers.Flatten
Dense = tf.keras.layers.Dense

# Définir un CNN pour l'agent Pong
def create_pong_model():
  model = tf.keras.models.Sequential([
    # Couches convolutionnelles
    # Premièrement, 16 filtres 7x7 avec 4x4 stride
    Conv2D(filters=16, kernel_size=7, strides=4),

    # A FAIRE : définir des couches convolutionnelles avec 32 filtres 5x5 et 2x2 stride
    Conv2D('''A FAIRE'''),

    # A FAIRE : définir des couches convolutionnelles avec 48 filtres 3x3 et 2x2 stride
    Conv2D('''A FAIRE'''),

    Flatten(),
    
    # Couche et sortie entièrement connectées
    Dense(units=64, activation='relu'),
    # A FAIRE : définir la dimension de sortie de la dernière couche dense. 
    # Faites attention à l'espace dans lequel l'agent doit agir
    Dense('''A FAIRE''')
  
  ])
  return model

pong_model = create_pong_model()

Puisque nous avons déjà défini la fonction d'action, `choose_action(model, observation)`, nous n'avons pas besoin de la définir à nouveau. Au lieu de cela, nous pourrons la réutiliser plus tard en passant dans notre nouveau modèle que nous venons de créer, `pong_model`. C'est génial parce que notre fonction d'action fournit une méthode modulaire et généralisable pour toutes sortes d'agents RL !

## 3.8 Fonctions spécifiques à Pong

Dans la partie 1 (Cartpole), nous avons implémenté quelques fonctions et classes clés pour construire et former notre agent RL -- `choose_action(model, observation)` et la classe `Memory`, par exemple. Cependant, en nous préparant à les appliquer à un nouveau jeu comme Pong, nous pourrions avoir besoin d'apporter quelques légères modifications. 

A savoir, nous devons penser à ce qui se passe quand un jeu se termine. Dans Pong, nous savons qu'une partie est terminée si la récompense est de +1 (nous avons gagné !) ou de -1 (nous avons malheureusement perdu). Sinon, nous nous attendons à ce que la récompense à un moment donné soit de zéro - les joueurs (ou les agents) ne font que s'affronter. Ainsi, après la fin d'une partie, nous devrons remettre la récompense à zéro à la fin de la partie. Cela entraînera une modification de la fonction de récompense.

In [0]:
### Fonction de récompense de Pong ###

# Calculer les récompenses normalisées et réduites pour Pong (c'est-à-dire le retour)
# Arguments:
#   rewards: récompense à chaque pas de temps dans l'épisode
#   gamma: facteur d'actualisation. Notez que l'augmentation à 0,99 -- le taux d'amortissement sera plus lent.
# Returns:
#   récompense réduite normalisée
def discount_rewards(rewards, gamma=0.99): 
  discounted_rewards = np.zeros_like(rewards)
  R = 0
  for t in reversed(range(0, len(rewards))):
      # NOUVEAU : Remettez la somme à zéro si la récompense n'est pas de 0 (le jeu est terminé !)
      if rewards[t] != 0:
        R = 0
      # mettre à jour le montant total de la récompense réduite comme auparavant
      R = R * gamma + rewards[t]
      discounted_rewards[t] = R
      
  return normalize(discounted_rewards)

En outre, nous devons tenir compte de la nature des observations dans l'environnement Pong et de la manière dont elles seront introduites dans notre réseau. Dans ce cas, nos observations sont des images. Avant d'entrer une image dans notre réseau, nous allons faire un peu de prétraitement pour la recadrer et la mettre à l'échelle, nettoyer les couleurs de fond pour obtenir une couleur unique et régler les éléments importants du jeu sur une couleur unique. Utilisons cette fonction pour visualiser à quoi pourrait ressembler une observation avant et après le prétraitement.

In [0]:
observation = env.reset()
for i in range(30):
  observation, _,_,_ = env.step(0)
observation_pp = mdl.lab3.preprocess_pong(observation)

f = plt.figure(figsize=(10,3))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax.imshow(observation); ax.grid(False);
ax2.imshow(np.squeeze(observation_pp)); ax2.grid(False); plt.title('Preprocessed Observation');

Que remarquez-vous ? En quoi ces changements peuvent-ils être importants pour la formation de notre algorithme RL ?

## 3.9 Apprentissage de Pong

Nous sommes maintenant prêts à commencer l'apprentissage de notre algorithme RL et de notre agent pour le jeu de Pong ! Nous avons déjà défini notre fonction de perte avec `compute_loss`, qui utilise l'apprentissage par gradient de politique, ainsi que notre étape de rétropropagation avec `train_step` qui est magnifique ! Nous utiliserons ces fonctions pour exécuter l'apprentissage de l'agent Pong. Passons en revue le bloc d'apprentissage.

Dans Pong, plutôt que d'alimenter notre réseau une image à la fois, on peut en fait améliorer les performances en entrant la différence entre deux observations consécutives, ce qui nous donne vraiment des informations sur le mouvement entre les images -- comment le jeu évolue. Nous allons d'abord prétraiter l'observation brute, "x", puis nous calculerons la différence avec l'image que nous avons vue une fois auparavant. 

Ce changement d'observation sera propagé dans notre agent Pong, le modèle de réseau CNN, qui prédira alors la prochaine action à entreprendre en fonction de cette observation. La récompense brute sera calculée, et l'observation, l'action et la récompense seront enregistrées dans la mémoire. Cela se poursuivra jusqu'à la fin d'un épisode d'entraînement, c'est-à-dire d'un jeu.

Ensuite, nous calculerons les récompenses réduites et utiliserons ces informations pour exécuter une étape d'entraînement. La mémoire sera effacée, et nous recommencerons !

Exécutons le bloc de code pour former notre agent Pong. Notez que l'apprentissage prendra un certain temps (estimé à au moins quelques heures). Nous allons à nouveau visualiser l'évolution de la récompense totale en fonction de la formation pour avoir une idée de la façon dont l'agent apprend.

In [0]:
### Former Pong ###

# Hyperparamètres
learning_rate=1e-4
MAX_ITERS = 10000 # augmenter le nombre maximum d'épisodes, car Pong est plus complexe !

# Modèle et optimiseur
pong_model = create_pong_model()
optimizer = tf.keras.optimizers.Adam(learning_rate)

# plotting
smoothed_reward = mdl.util.LossHistory(smoothing_factor=0.9)
plotter = mdl.util.PeriodicPlotter(sec=5, xlabel='Iterations', ylabel='Rewards')
memory = Memory()

for i_episode in range(MAX_ITERS):

  plotter.plot(smoothed_reward.get())

  # Relancer l'environnement
  observation = env.reset()
  previous_frame = mdl.lab3.preprocess_pong(observation)

  while True:
      # Pré-processer l'image 
      current_frame = mdl.lab3.preprocess_pong(observation)
      
      '''A FAIRE : déterminer le changement d'observation
      Indice : c'est la différence entre les deux dernières images'''
      obs_change = # A FAIRE
      
      '''A FAIRE : choisir une action pour le modèle de pong, en utilisant la différence de cadre, et évaluer'''
      action = # A FAIRE
      # Faire l'action choisie
      next_observation, reward, done, info = env.step(action)

      '''A FAIRE : sauver la différence d'image observée, l'action qui a été entreprise et la récompense qui en résulte !'''
      memory.add_to_memory('''A FAIRE''', '''A FAIRE''', '''A FAIRE''')
      
      # L'épisode est-il terminé ? Vous êtes-vous planté ou avez-vous si bien réussi que vous avez terminé ?
      if done:
          # déterminer la récompense totale et en conserver une trace
          total_reward = sum(memory.rewards)
          smoothed_reward.append( total_reward )

          # commencer l'apprentissage
          train_step(pong_model, 
                     optimizer, 
                     observations = np.stack(memory.observations, 0), 
                     actions = np.array(memory.actions),
                     discounted_rewards = discount_rewards(memory.rewards))
          
          memory.clear()
          break

      observation = next_observation
      previous_frame = current_frame

Enfin, nous pouvons mettre notre agent formé à l'épreuve ! Il jouera dans un environnement de Pong nouvellement instancié contre "l'ordinateur", un système d'IA de base pour le Pong. Votre agent joue le rôle de la raquette verte. Regardons la retransmission instantanée du match !

In [0]:
saved_pong = mdl.lab3.save_video_of_model(
    pong_model, "Pong-v0", obs_diff=True, 
    pp_fn=mdl.lab3.preprocess_pong)
mdl.lab3.play_video(saved_pong)

## 3.10 Conclusion

Ça y est ! Félicitations pour avoir formé deux agents de RL et les avoir mis à l'épreuve ! Nous vous encourageons à considérer les points suivants :

* Comment l'agent se comporte-t-il ?
* Pourriez-vous le former pendant une période plus courte tout en obtenant de bons résultats ?
* Pensez-vous qu'une formation plus longue serait encore plus utile ? 
* Comment la complexité de Pong par rapport au Cartpole modifie-t-elle le rythme d'apprentissage de l'agent et ses performances ? 
* Quelles sont les choses que vous pourriez changer à propos de l'agent ou du processus d'apprentissage pour améliorer potentiellement ses performances ?

Si vous voulez aller plus loin, essayez d'optimiser votre modèle pour obtenir les meilleures performances ! ** [Envoyez-nous](mailto:introtodeeplearning-staff@mit.edu) une copie de votre notebook avec la formation Pong exécutée ET une vidéo sauvegardée de votre agent Pong en compétition ! Nous donnerons des prix aux meilleurs joueurs !